In [59]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg
# import cv2
from unified_planning.shortcuts import *
import ast

In [61]:
with open("../plans/bricks.txt", "r") as file:
    content = file.read()
    # print(content)
    fixed = content.replace('[[[', '[').replace('[[','[').replace(']]]',']').replace(']]',']')
    # print(fixed)
    fixed_array = ast.literal_eval(fixed)

    brick_dictionary = {}
    for array in fixed_array:
        print(array)
        brick_dictionary[array[1]] = [array[0], array[2], array[3], array[4]]

[0, '1', ' Brick', ' Brown', ' 2x4']
[1, '2', ' Brick', ' Green', ' 2x4']
[1, '3', ' Brick', ' Red', ' 2x2']
[2, '1', ' Brick', ' Brown', ' 2x4']
[2, '4', ' Brick', ' Tan', ' 2x2']
[2, '5', ' Brick', ' White', ' 1x1']
[3, '3', ' Brick', ' Red', ' 2x2']
[3, '6', ' Plate', ' Transparent', ' 1x2']
[3, '7', ' Plate', ' Transparent', ' 1x1']
[3, '8', ' Plate', ' White', ' 1x1']
[3, '9', ' Plate', ' White', ' 1x2']
[4, '1', ' Brick', ' Brown', ' 2x4']
[4, '10', ' Brick', ' White', ' 1x1']
[4, '11', ' Plate', ' Transparent', ' 1x1']
[4, '12', ' Plate', ' White', ' 1x2']
[5, '3', ' Brick', ' Red', ' 2x2']
[5, '10', ' Brick', ' White', ' 1x1']
[5, '13', ' Plate', ' White', ' 1x1']
[6, '3', ' Brick', ' Red', ' 2x2']
[6, '14', ' Brick', ' White', ' 1x3']
[7, '14', ' Brick', ' White', ' 1x3']
[7, '15', ' Plate', ' White', ' 1x1']
[8, '16', ' Brick', ' White', ' 1x1']
[8, '17', ' Plate', ' Black', ' 1x1']
[9, '1', ' Brick', ' Brown', ' 2x4']
[9, '3', ' Brick', ' Red', ' 2x2']
[9, '18', ' Brick', ' 

In [62]:
#Unified Planning Object Locations

Location = UserType('Location')
Bricks = UserType('Bricks')
brick_at = unified_planning.model.Fluent('brick_at', BoolType(), l=Location)



In [63]:
#Create Grid Squares
locs = []
count = 0
for y in range(5):
    for x in range(5):
        loc = f'loc-{count}'
        locs.append(loc)
        count+=1
        print(loc)
locations = [unified_planning.model.Object(l,Location) for l in locs]


loc-0
loc-1
loc-2
loc-3
loc-4
loc-5
loc-6
loc-7
loc-8
loc-9
loc-10
loc-11
loc-12
loc-13
loc-14
loc-15
loc-16
loc-17
loc-18
loc-19
loc-20
loc-21
loc-22
loc-23
loc-24


In [77]:
#Create Brick Objects

seperate_bricks = brick_dictionary.keys()
bricks = [unified_planning.model.Object(b,Bricks) for b in seperate_bricks]




In [65]:
#States

brick_at = unified_planning.model.Fluent('brick_at', BoolType(), brick = Bricks, loc = Location)
on = unified_planning.model.Fluent('on', BoolType(), base_brick = Bricks, stack_brick = Bricks)
clear = unified_planning.model.Fluent('clear',BoolType(), brick = Bricks)



In [66]:
#Create Problem

problem = unified_planning.model.Problem('LegoScatter')

In [67]:
problem.add_fluent(brick_at, default_initial_value=False)
problem.add_fluent(on, default_initial_value=False)
problem.add_fluent(clear, default_initial_value=False)

bool clear[brick=Bricks]

In [68]:
#Move from one grid location to another grid square

move = unified_planning.model.InstantaneousAction('move',brick = Bricks, l_from=Location, l_to=Location)
curr_brick = move.parameter('brick')
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(brick_at(curr_brick, l_from))
move.add_precondition(clear(curr_brick))
move.add_effect(brick_at(curr_brick, l_from), False)
move.add_effect(brick_at(curr_brick, l_to), True)
problem.add_action(move)

In [69]:
#If two bricks are in the same location, stack one on top of the other

move_on = unified_planning.model.InstantaneousAction('move_on',base_brick = Bricks, stack_brick = Bricks, l_curr=Location)
base_brick = move_on.parameter('base_brick')
l_curr = move_on.parameter('l_curr')
stack_brick = move_on.parameter('stack_brick')
move_on.add_precondition(brick_at(base_brick, l_curr))
move_on.add_precondition(brick_at(stack_brick, l_curr))
move_on.add_precondition(clear(base_brick))
move_on.add_effect(on(stack_brick,base_brick ), True)
move_on.add_effect(clear(base_brick),False)
problem.add_action(move_on)

In [70]:
#Reversed - take a block off the top of the tower

move_off = unified_planning.model.InstantaneousAction('move_off',base_brick = Bricks, stack_brick = Bricks, l_curr=Location)
base_brick = move_off.parameter('base_brick')
l_curr = move_off.parameter('l_curr')
stack_brick = move_off.parameter('stack_brick')
move_off.add_precondition(brick_at(base_brick, l_curr))
move_off.add_precondition(brick_at(stack_brick, l_curr))
move_off.add_precondition(on(stack_brick,base_brick))
move_off.add_precondition(clear(stack_brick))
move_off.add_effect(on(stack_brick,base_brick ), False)
move_off.add_effect(clear(base_brick),True)
problem.add_action(move_off)

In [84]:
#Initialise Locations

#DICTIONARY TAKES FORM:

# KEY IS THE UNIQUE IDENTIFIER
# VALUE IS [LOCATION,TYPE,COLOUR,SIZE]

count = 0
for key, value in brick_dictionary.items():
    # print(key,value)
    loc = Object(f'loc-{value[0]}',Location)
    # print(loc)
    temp_brick = Object(key,Bricks)
    problem.set_initial_value(brick_at(temp_brick,loc),True)
    problem.set_initial_value(clear(temp_brick),True)


# for i in range(len(bricks)):
#     loc = Object(f'loc-{final_locations[i+1][0]},{final_locations[i+1][1]}',Location)
#     print(loc)
    # problem.set_initial_value(brick_at(bricks[i],loc),True)
    # problem.set_initial_value(clear(bricks[i]), True)

In [93]:
 #INITIALISING LEFT RIGHT BEHING ON ETC

with open("ITEM_POSITIONS.txt", "r") as file:
    for line in file:
        if not line.lstrip().startswith("["):
            # print(line, end="")  # Use end="" to avoid double newlines
            parts = line.strip().split()
            b1, relationship, b2 = parts
            if relationship == 'on':
                base_brick = Object(b2,Bricks)
                stack_brick = Object(b1,Bricks)
                problem.set_initial_value(on(base_brick,stack_brick),True)
                problem.set_initial_value(clear(base_brick),False)

In [94]:
 print(problem)

problem name = LegoScatter

types = [Bricks, Location]

fluents = [
  bool brick_at[brick=Bricks, loc=Location]
  bool on[base_brick=Bricks, stack_brick=Bricks]
  bool clear[brick=Bricks]
]

actions = [
  action move(Bricks brick, Location l_from, Location l_to) {
    preconditions = [
      brick_at(brick, l_from)
      clear(brick)
    ]
    effects = [
      brick_at(brick, l_from) := false
      brick_at(brick, l_to) := true
    ]
  }
  action move_on(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      clear(base_brick)
    ]
    effects = [
      on(stack_brick, base_brick) := true
      clear(base_brick) := false
    ]
  }
  action move_off(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      on(stack_brick, base_brick)
      clear(stack_brick)
    ]
    effects = [
      on

In [97]:
with open("goalstates.txt", "r") as file:
    for line in file:
        parts = line.strip().split()
        b1, relationship, b2 = parts
        if relationship == 'on':
            base_brick = Object(b1,Bricks)
            stack_brick = Object(b2,Bricks)
            problem.add_goal(on(base_brick,stack_brick))

In [72]:
# #Hardcoded Goals
# Stack all corresponding colours
# problem.add_goal(on(Object("P1",Bricks),Object("P2",Bricks)))
# problem.add_goal(on(Object("P2",Bricks),Object("P3",Bricks)))
# problem.add_goal(on(Object("P3",Bricks),Object("P4",Bricks)))
# problem.add_goal(on(Object("R1",Bricks),Object("R2",Bricks)))
# problem.add_goal(on(Object("R2",Bricks),Object("R3",Bricks)))
# problem.add_goal(on(Object("G1",Bricks),Object("G2",Bricks)))

In [98]:
print(problem)

problem name = LegoScatter

types = [Bricks, Location]

fluents = [
  bool brick_at[brick=Bricks, loc=Location]
  bool on[base_brick=Bricks, stack_brick=Bricks]
  bool clear[brick=Bricks]
]

actions = [
  action move(Bricks brick, Location l_from, Location l_to) {
    preconditions = [
      brick_at(brick, l_from)
      clear(brick)
    ]
    effects = [
      brick_at(brick, l_from) := false
      brick_at(brick, l_to) := true
    ]
  }
  action move_on(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      clear(base_brick)
    ]
    effects = [
      on(stack_brick, base_brick) := true
      clear(base_brick) := false
    ]
  }
  action move_off(Bricks base_brick, Bricks stack_brick, Location l_curr) {
    preconditions = [
      brick_at(base_brick, l_curr)
      brick_at(stack_brick, l_curr)
      on(stack_brick, base_brick)
      clear(stack_brick)
    ]
    effects = [
      on